# Project
This is the main file for project 2 in COMP 472.

**Team:** Deus Ex Machina

**Member(s):** Sobhan Mehrpour Kevishahi - 40122438

**Github Repository :** [https://github.com/Sobhan-M/comp472-project2](https://github.com/Sobhan-M/comp472-project2)

## Implementation Of Algorithms
In this section I will be implementing the different search algorithms as methods. The implemented classes and functions have been included in separate Python files and are used to abstract away tedious aspects of the implementation.

In [1]:
# I realize this way of importing is not good practice, but I know there aren't any conflicts so I'm doing it anyway.

from car import *
from grid import *
from position import *
from node import *
from priorityqueue import *
from reporter import *
from heuristics import *

import numpy as np

In [2]:
example1 = "IIB...C.BHHHC.AAD.....D.EEGGGF.....F"
example2 = "C.B...C.BHHHAADD........EEGGGF.....F"
example3 = "...GF...BGF.AABCF....CDD...C....EE.."
example4 = "....F...B.F.AABCF....C.....C....EE.."
example5 = "BBBJCCH..J.KHAAJ.K..IDDLEEI..L....GG H3 K4 J3"

examples = [example1, example2, example3, example4, example5]

### Uniform Cost Search
This is the implementation of uniform cost search. This algorithm only uses the cost (distance from root node in this case) when choosing which node to expand.

In [3]:
def uniformCostSearch(puzzleLine:str, reportNumber=0, outputFileName="ucs"):
	"""
	Performs uniform cost search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: n.cost()) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: n.cost(), lambda n: 0)

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Update node if needed.
				if child.cost() < openList.getValue(child).cost():
					openList.updateValue(child)
			else:
				openList.insert(child)

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
	searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

	with open(solutionFileName, "w") as file:
		file.write(report)

	with open(searchFileName, "w") as file:
		file.write("\n".join(reporter.searchPath))

	return goalNode

In [4]:
# for i in range(5):
# 	uniformCostSearch(examples[i], i+1, "example-reports/ucs-example")

### Greedy Best First Search
In this implementation we use a heuristic to determine which nodes to explore. This does not guarantee an optimal solution.

In [5]:
def greedyBestFirstSearch(puzzleLine:str, heuristic, reportNumber=0, outputFileName="gbfs"):
	"""
	Performs greedy best first search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: heuristic(n)) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if child.isGoal():
				goalNode = child
				break

			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Avoid visiting a child node already in the open list. Heuristics don't change.
				continue

			openList.insert(child)

		if goalNode is not None:
			# Counting the child if it's the goal node.
			reporter.countVisit()
			reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))
			break

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
	searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

	with open(solutionFileName, "w") as file:
		file.write(report)

	with open(searchFileName, "w") as file:
		file.write("\n".join(reporter.searchPath))

	return goalNode

	

In [6]:
# for i in range(5):
# 	greedyBestFirstSearch(examples[i], h1, i+1, "example-reports/gbfs-example")